In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="Query_analysis"

In [2]:
# LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# Load documents
We can use the YouTubeLoader to load transcripts of a few LangChain videos:

In [3]:
from langchain_community.document_loaders import YoutubeLoader

urls = [
    "https://www.youtube.com/watch?v=HAn9vnJy6S4",
    "https://www.youtube.com/watch?v=dA1cHGACXCo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())

# Add some additional metadata: what year the video was published

In [4]:
import datetime

for doc in docs:
    doc.metadata["publish_year"] = int(
        datetime.datetime.strptime(
            doc.metadata["publish_date"], "%Y-%m-%d %H:%M:%S"
        ).strftime("%Y")
    )

Here are the titles of the videos we've loaded:

In [5]:
[doc.metadata["title"] for doc in docs]

['OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows']

Here's the metadata associated with each video. We can see that each document also has a title, view count, publication date, and length:



In [6]:
docs[0].metadata

{'source': 'HAn9vnJy6S4',
 'title': 'OpenGPTs',
 'description': 'Unknown',
 'view_count': 9253,
 'thumbnail_url': 'https://i.ytimg.com/vi/HAn9vnJy6S4/hq720.jpg',
 'publish_date': '2024-01-31 00:00:00',
 'length': 1530,
 'author': 'LangChain',
 'publish_year': 2024}

And here's a sample from a document's contents:



In [7]:
docs[0].page_content[:500]

"hello today I want to talk about open gpts open gpts is a project that we built here at linkchain uh that replicates the GPT store in a few ways so it creates uh end user-facing friendly interface to create different Bots and these Bots can have access to different tools and they can uh be given files to retrieve things over and basically it's a way to create a variety of bots and expose the configuration of these Bots to end users it's all open source um it can be used with open AI it can be us"

# Indexing documents
Whenever we perform retrieval we need to create an index of documents that we can query. We'll use a vector store to index our documents, and we'll chunk them first to make our retrievals more concise and precise:

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
chunked_docs = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(
    chunked_docs,
    embeddings,
)

# Retrieval without query analysis
We can perform similarity search on a user question directly to find chunks relevant to the question:





In [9]:
search_results = vectorstore.similarity_search("how do I build a RAG agent")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

OpenGPTs
hardcoded that it will always do a retrieval step here the assistant decides whether to do a retrieval step or not sometimes this is good sometimes this is bad sometimes it you don't need to do a retrieval step when I said hi it didn't need to call it tool um but other times you know the the llm might mess up and not realize that it needs to do a retrieval step and so the rag bot will always do a retrieval step so it's more focused there because this is also a simpler architecture so it's always


This works pretty well! Our first result is quite relevant to the question.

What if we wanted to search for results from a specific time period?

In [10]:
search_results = vectorstore.similarity_search("videos on RAG published in 2023")
print(search_results[0].metadata["title"])
print(search_results[0].metadata["publish_date"])
print(search_results[0].page_content[:500])

OpenGPTs
2024-01-31 00:00:00
hardcoded that it will always do a retrieval step here the assistant decides whether to do a retrieval step or not sometimes this is good sometimes this is bad sometimes it you don't need to do a retrieval step when I said hi it didn't need to call it tool um but other times you know the the llm might mess up and not realize that it needs to do a retrieval step and so the rag bot will always do a retrieval step so it's more focused there because this is also a simpler architecture so it's always


Our first result is from 2024 (despite us asking for videos from 2023), and not very relevant to the input. Since we're just searching against document contents, there's no way for the results to be filtered on any document attributes.

This is just one failure mode that can arise. Let's now take a look at how a basic form of query analysis can fix it!

# Query analysis
We can use query analysis to improve the results of retrieval. 

This will involve defining a query schema that contains some date filters and use a function-calling model to convert a user question into a structured queries.

# Query schema (Data Model)
In this case we'll have explicit min and max attributes for publication date so that it can be filtered on.


Defines a Search data model with two fields: 
1. **query**, a required string used for searching video transcripts
2. **publish_year**, an optional integer indicating the year the video was published. 
 

The model uses Pydantic for automatic validation and type-checking of these fields.



- **Search(BaseModel)**: This class inherits from BaseModel, making it a Pydantic model. 
  This means that instances of Search will automatically validate the types and constraints of the fields.

- **query**: This is a required field of type str. 
  The Field(...) indicates that this field is mandatory (as ... is a placeholder indicating that a value must be provided). 
  The description parameter provides a human-readable description of this field, which is useful for documentation or validation messages.

- **publish_year**: This is an optional field of type int. 
  Since it's wrapped in Optional[int], it can either be an integer representing the year or None if the year is not specified. 
  The Field(None) sets the default value to None, and the description explains that this field represents the year the video was published.

In [12]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    query: str = Field(..., description="Similarity search query applied to video transcripts.",)
    publish_year: Optional[int] = Field(None, description="Year video was published")

### Query generation
To convert user questions to structured queries we'll make use of OpenAI's tool-calling API. 
Specifically we'll use the new ChatModel.with_structured_output() constructor to handle passing the schema to the model and parsing the output.

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [ ]:

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(Search)

In [ ]:
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [14]:
query_analyzer.invoke("how do I build a RAG agent")

Search(query='build RAG agent', publish_year=None)

In [15]:
query_analyzer.invoke("videos on RAG published in 2023")

Search(query='RAG', publish_year=2023)

### Retrieval with query analysis
Our query analysis looks pretty good; now let's try using our generated queries to actually perform retrieval.

Note: in our example, we specified tool_choice="Search". 

This will force the LLM to call one - and only one - tool, meaning that we will always have one optimized query to look up. 
Note that this is not always the case - see other guides for how to deal with situations when no - or multiple - optmized queries are returned.



This code defines a function named retrieval that takes a Search object as input and returns a list of Document objects. Here's a detailed explanation:

* def retrieval(search: Search): This defines a function named retrieval that takes one argument, search, which is expected to be an instance of the Search class defined earlier.
* -> List[Document]: This is a type hint indicating that the function returns a list of Document objects.

In [16]:
from typing import List

from langchain_core.documents import Document

# Create a retrieval filter (Chroma Specific)

In [22]:
def retrieval(search: Search) -> List[Document]:
    if search.publish_year is not None:
        # This is syntax specific to Chroma,
        # the vector database we are using.
        _filter = {"publish_year": {"$eq": search.publish_year}}
    else:
        _filter = None
    return vectorstore.similarity_search(search.query, filter=_filter)

# Chain together the filter and the retriever

In [24]:
retrieval_chain = query_analyzer | retrieval

In [25]:
results = retrieval_chain.invoke("RAG tutorial published in 2024")

In [26]:
[(doc.metadata["title"], doc.metadata["publish_date"]) for doc in results]

[('OpenGPTs', '2024-01-31 00:00:00'),
 ('OpenGPTs', '2024-01-31 00:00:00'),
 ('OpenGPTs', '2024-01-31 00:00:00'),
 ('Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
  '2024-01-26 00:00:00')]